# Querying the stackoverflow dataset using BigQuery

This notebook was run on Kaggle as it's easier to get permission there.

In [5]:
import bq_helper
from bq_helper import BigQueryHelper
# https://www.kaggle.com/sohier/introduction-to-the-bq-helper-package
stackOverflow = bq_helper.BigQueryHelper(active_project="bigquery-public-data",
                                   dataset_name="stackoverflow")

## Print out tables in the stackoverflow dataset

In [12]:
bq_assistant = BigQueryHelper("bigquery-public-data", "stackoverflow")
tables = bq_assistant.list_tables()
tables

['badges',
 'comments',
 'post_history',
 'post_links',
 'posts_answers',
 'posts_moderator_nomination',
 'posts_orphaned_tag_wiki',
 'posts_privilege_wiki',
 'posts_questions',
 'posts_tag_wiki',
 'posts_tag_wiki_excerpt',
 'posts_wiki_placeholder',
 'stackoverflow_posts',
 'tags',
 'users',
 'votes']

## Get columns per table

In [ ]:
for table in tables:
    index = bq_assistant.head(table, num_rows=0)
    print(table, index.columns.tolist(),'\n')

badges ['id', 'name', 'date', 'user_id', 'class', 'tag_based'] 

comments ['id', 'text', 'creation_date', 'post_id', 'user_id', 'user_display_name', 'score'] 

post_history ['id', 'creation_date', 'post_id', 'post_history_type_id', 'revision_guid', 'user_id', 'text', 'comment'] 

post_links ['id', 'creation_date', 'link_type_id', 'post_id', 'related_post_id'] 

posts_answers ['id', 'title', 'body', 'accepted_answer_id', 'answer_count', 'comment_count', 'community_owned_date', 'creation_date', 'favorite_count', 'last_activity_date', 'last_edit_date', 'last_editor_display_name', 'last_editor_user_id', 'owner_display_name', 'owner_user_id', 'parent_id', 'post_type_id', 'score', 'tags', 'view_count'] 

posts_moderator_nomination ['id', 'title', 'body', 'accepted_answer_id', 'answer_count', 'comment_count', 'community_owned_date', 'creation_date', 'favorite_count', 'last_activity_date', 'last_edit_date', 'last_editor_display_name', 'last_editor_user_id', 'owner_display_name', 'owner_user_id

## Get posts with comments and answers

- Scores are above > 0
- Limited

In [50]:
# bigquery-public-data.stackoverflow. must be infront of every table

query2 = """
WITH DockerPosts AS (
  SELECT id, title, body, tags, creation_date
  FROM bigquery-public-data.stackoverflow.stackoverflow_posts
  WHERE 
    (LOWER(title) LIKE '%docker%' OR
     LOWER(tags) LIKE '%docker%')
    AND score > 0  
  LIMIT 500000  
)

SELECT * FROM (
  SELECT 
    dp.id AS post_id,
    dp.title AS post_title,  -- This is STRING
    dp.creation_date AS post_creation_date,
    'Post' AS content_type,
    dp.body AS content,
    CAST(NULL AS STRING) AS parent_post_id  -- Ensure STRING type
  FROM DockerPosts dp

  UNION ALL

  SELECT 
    pa.id AS post_id,
    CAST(NULL AS STRING) AS post_title,  -- Convert to STRING
    pa.creation_date AS post_creation_date,
    'Answer' AS content_type,
    pa.body AS content,
    CAST(pa.parent_id AS STRING) AS parent_post_id  -- Ensure STRING type
  FROM DockerPosts dp
  JOIN bigquery-public-data.stackoverflow.posts_answers pa 
    ON dp.id = pa.parent_id
  WHERE pa.score > 0  
) 

UNION ALL

SELECT * FROM (
  SELECT 
    c.id AS post_id,
    CAST(NULL AS STRING) AS post_title,  -- Convert to STRING
    c.creation_date AS post_creation_date,
    'Comment' AS content_type,
    c.text AS content,
    CAST(c.post_id AS STRING) AS parent_post_id  -- Ensure STRING type
  FROM DockerPosts dp
  JOIN bigquery-public-data.stackoverflow.comments c 
    ON dp.id = c.post_id
  WHERE c.score > 0  
) 

ORDER BY post_creation_date;


"""
response1 = stackOverflow.query_to_pandas_safe(query2,max_gb_scanned=71)
print(response1)
response1.head(10)

         post_id      ...       parent_post_id
0       15644643      ...                 None
1       15645715      ...             15644643
2       15693153      ...                 None
3       15693403      ...                 None
4       15693440      ...             15693153
5       15697996      ...             15693403
6       15700943      ...                 None
7       22395011      ...             15700943
8       22398292      ...             15700943
9       15768121      ...                 None
10      15789067      ...             15768121
11      16047306      ...                 None
12      16047960      ...                 None
13      16048358      ...             16047306
14      16084741      ...                 None
15      16176058      ...             16084741
16      16182968      ...                 None
17      16182969      ...             16182968
18      16232661      ...                 None
19      16284004      ...                 None
20      16296

,post_id,post_title,post_creation_date,content_type,content,parent_post_id
0,15644643,When will Docker be launched?,2013-03-26 18:24:32.770000+00:00,Post,<p>I need a tool to encapsulate heterogeneous ...,None
1,15645715,None,2013-03-26 19:26:19.197000+00:00,Answer,<p>It has been launched few minutes ago! <a hr...,15644643
2,15693153,Mounting directory from parent system to conta...,2013-03-28 22:17:12.170000+00:00,Post,<p>How to mount directory from parent system t...,None
3,15693403,Firewall for the container in docker,2013-03-28 22:39:55.770000+00:00,Post,<p>How to know network interface of the contai...,None
4,15693440,None,2013-03-28 22:42:46.153000+00:00,Answer,<p><strong>*Update - see answer below. this is...,15693153
5,15697996,None,2013-03-29 05:43:20.217000+00:00,Answer,"<p>Ok, I asked that question on IRC. If you ne...",15693403
6,15700943,Backup a running Docker container?,2013-03-29 09:54:32.890000+00:00,Post,<p>Is it possible to backup a running Docker c...,None
7,22395011,None,2013-04-02 06:35:06.453000+00:00,Comment,"Hi Slava, sorry that your question was closed....",15700943
8,22398292,None,2013-04-02 08:40:16.787000+00:00,Comment,@SolomonHykes One more clarification - will it...,15700943
9,15768121,docker: net: no such interface,2013-04-02 15:30:47.347000+00:00,Post,<p>I have just installed docker on a fedora 18...,None
